In [ ]:
import json
import requests
import pandas as pd
import os
import sys
import csv
import os.path
import pandas as pd
from os import listdir
from os.path import isfile, join
import glob
import numpy as np
import random

# constant
AUDIO_FEATURES_OUTPUT_FILE_PATH='/home/common/data/audio_feature/non_liked_songs_audio_features_{fid}.csv'
TRACKS_DATA_PATH = '/home/common/data/tracks/*.csv'

# spotify end points 
get_audio_analysis_url = "https://api.spotify.com/v1/audio-features"

# get the access token from environment
access_token = "Bearer " + (os.environ['SPOTIFY_TOKEN'])

In [ ]:
# Read the unique tracks
files = glob.glob(TRACKS_DATA_PATH)
list = []
for filename in files:
    dft = pd.read_csv(filename,header=0)
    list.append(dft)
df = (pd.concat(list, axis=0, ignore_index=True)[['id']]).id.unique()
tracks = df.tolist()
print("Total tracks ",len(tracks))

In [ ]:
def write_audio_features(tlist,cnt):
    t = ",".join(tlist)
    print(t)
    r = requests.get(url=get_audio_analysis_url,
           params={"ids":t},
           headers={"Content-Type": "application/json","Accept": "application/json","Authorization": access_token})
    print(r.status_code)
    data = r.json()
    df = pd.DataFrame(data['audio_features'])
    file_path = AUDIO_FEATURES_OUTPUT_FILE_PATH.format(fid=cnt)
    print(file_path)
    df.to_csv(file_path)

ids_list=[]
cnt=1
for i in range(len(tracks)):
    if i % 100 == 0:
        write_audio_features(ids_list,cnt)
        ids_list.clear()
        cnt=cnt+1
        print("Written the data ...")
    else:
        ids_list.append(tracks[i].strip())
write_audio_features(ids_list,cnt)

In [ ]:
GLOB_ALL_AUDIO_FEATURES_OUTPUT_FILE_PATH='/home/common/data/audio_feature/non_liked_songs_audio_features_*.csv'
ALL_AUDIO_FEATURES_OUTPUT_FILE_PATH='/home/common/data/audio_feature/all_non_liked_songs_audio_features.csv'
files = glob.glob(GLOB_ALL_AUDIO_FEATURES_OUTPUT_FILE_PATH)
list = []
for filename in files:
    print(filename)
    dft = pd.read_csv(filename,header=0)
    list.append(dft)
df = pd.concat(list, axis=0, ignore_index=True).drop(['Unnamed: 0'],axis = 1)
df.to_csv(ALL_AUDIO_FEATURES_OUTPUT_FILE_PATH)